In [1]:
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
from Models.selective_sequential import *
from Loss.triplet_regularized import *
from session import *
from LR_Schedule.cyclical import Cyclical
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
from session import LossMeter, EvalModel
from Layers.flatten import Flatten
from torch.utils.tensorboard import SummaryWriter

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/drake/anaconda3/envs/.

In [2]:
%load_ext autoreload
%autoreload 2

torch.cuda.set_device(0); torch.backends.cudnn.benchmark=True;

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/torch/cuda/__init__.py:135: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST('/media/drake/MX500/Datasets/mnist/train', download=True, train=True, transform=transform)
partial_trainset = torch.utils.data.dataset.Subset(trainset, np.arange(500))

valset = datasets.MNIST('/media/drake/MX500/Datasets/mnist/test', download=True, train=False, transform=transform)
partial_valset = torch.utils.data.dataset.Subset(valset, np.arange(500))

trainloader = torch.utils.data.DataLoader(partial_trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(partial_valset, batch_size=64, shuffle=False)

In [4]:
select = ['max1', 'act1', 'out']

def make_model():
    return SelectiveSequential(
    select,
    {'conv64': nn.Conv2d(1, 64, kernel_size=5, padding=2),
     'act64': nn.ReLU(True),
     
     'max1': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'conv192': nn.Conv2d(64, 192, kernel_size=5, padding=2),
     'act192': nn.ReLU(True),
    
     'max2': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'conv384': nn.Conv2d(192, 384, kernel_size=3, padding=1),
     'act384': nn.ReLU(True),
     
     'conv256a': nn.Conv2d(384, 256, kernel_size=3, padding=1),
     'act256a': nn.ReLU(True),
     
     'conv256b': nn.Conv2d(256, 256, kernel_size=3, padding=1),
     'act256b': nn.ReLU(True),
     
     'max3': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'flatten': Flatten(),
     'fc1': nn.Linear(3 * 3 * 256, 512),
     'act1': nn.ReLU(True),
     'fc2': nn.Linear(512, 512),
     'act2': nn.ReLU(True),
     'out': nn.Linear(512, 10)})

model = make_model()

In [5]:
criterion = TripletRegularizedMultiMarginLoss(.5, .5, select)
sess = Session(model, criterion, optim.Adam, 1e-4)

In [6]:
num_epochs = 50
validator = EmbeddingSpaceValidator(valloader, select, CustomOneHotAccuracy, 
                                    model_file="./triplet-reg.ckpt.tar")
lr_scheduler = CosAnneal(len(trainloader)*50, T_mult=1, lr_min=1e-6)
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])

In [ ]:
sess.train(schedule, 50)

In [ ]:
validator.plot()

In [ ]:
sess.load("./triplet-reg.ckpt.tar")

In [ ]:
total_valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)
total_validator = EmbeddingSpaceValidator(total_valloader, [], CustomOneHotAccuracy)

total_validator.run(sess)

In [ ]:
np.max(total_validator.val_accuracies), "Best accuracy without reg"

In [10]:
def train(lmbda, train_loader, val_loader):
    print(f"Training: {lmbda}")
    num_epochs = 20
    validator = EmbeddingSpaceValidator(val_loader, [], CustomOneHotAccuracy)
    lr_scheduler = CosAnneal(len(train_loader)*num_epochs, T_mult=1, lr_min=1e-6)
    schedule = TrainingSchedule(train_loader, [lr_scheduler, validator])
    criterion = TripletRegularizedMultiMarginLoss(lmbda, .5, select)
    sess = Session(make_model(), criterion, optim.Adam, 1e-4)
    sess.train(schedule, num_epochs)
    return np.max(validator.val_accuracies)

losses = {}

def search(lower, middle, upper, max_interval = .01, lower_acc=None, upper_acc=None):
    if (upper-lower < max_interval): return

    if lower_acc == None: 
        lower_acc = train(lower, trainloader, valloader)
        losses[lower] = lower_acc
    
    middle_acc = train(middle, trainloader, valloader)
    losses[middle] = middle_acc
    
    if upper_acc == None: 
        upper_acc = train(upper, trainloader, valloader)
        losses[upper] = upper_acc
        
    lower_mean = (lower_acc + middle_acc) / 2
    upper_mean = (upper_acc + middle_acc) / 2
    
    if lower_acc > upper_acc:
        search(lower, (middle-lower)/2, middle, max_interval, lower_acc, middle_acc)
    else:
        search(middle, (upper-middle)/2, upper, max_interval, middle_acc, upper_acc)

In [11]:
search(0, .5, 1)

Training: 0




val accuracy:  0.134 
train loss:  0.8985  train BCE :  0.8986 
valid loss:  0.8934  valid BCE :  0.8934




val accuracy:  0.144 
train loss:  0.8926  train BCE :  0.8881 
valid loss:  0.8763  valid BCE :  0.8763




val accuracy:  0.204 
train loss:  0.8729  train BCE :  0.8428 
valid loss:  0.7792  valid BCE :  0.7792




val accuracy:  0.254 
train loss:  0.8148  train BCE :  0.6927 
valid loss:  0.6637  valid BCE :  0.6637


...



val accuracy:  0.192 
train loss:  1.498  train BCE :  0.8954 
valid loss:  1.4988  valid BCE :  0.8904




val accuracy:  0.252 
train loss:  1.4823  train BCE :  0.8681 
valid loss:  1.4578  valid BCE :  0.8414




val accuracy:  0.51 
train loss:  1.4382  train BCE :  0.7531 
valid loss:  1.3473  valid BCE :  0.672




val accuracy:  0.492 
train loss:  1.372  train BCE :  0.5412 
valid loss:  1.2393  valid BCE :  0.5064


KeyboardInterrupt: 

In [12]:
losses

{0: 0.784,
 0.5: 0.938,
 1: 0.774,
 0.25: 0.916,
 0.125: 0.902,
 0.1875: 0.916,
 0.15625: 0.898,
 0.171875: 0.912,
 0.1640625: 0.886,
 0.16796875: 0.912,
 0.166015625: 0.908,
 0.1669921875: 0.91,
 0.16650390625: 0.922,
 0.166748046875: 0.92,
 0.1666259765625: 0.912,
 0.16668701171875: 0.91,
 0.166656494140625: 0.906,
 0.1666717529296875: 0.906,
 0.16666412353515625: 0.908,
 0.16666793823242188: 0.92,
 0.16666603088378906: 0.894,
 0.16666698455810547: 0.92,
 0.16666650772094727: 0.92,
 0.16666674613952637: 0.91,
 0.16666662693023682: 0.908,
 0.1666666865348816: 0.896,
 0.1666666567325592: 0.882,
 0.1666666716337204: 0.882,
 0.1666666641831398: 0.914,
 0.1666666679084301: 0.906,
 0.16666666604578495: 0.918,
 0.16666666697710752: 0.914,
 0.16666666651144624: 0.912,
 0.16666666674427688: 0.916,
 0.16666666662786156: 0.916,
 0.16666666668606922: 0.916,
 0.1666666666569654: 0.916,
 0.1666666666715173: 0.914,
 0.16666666666424135: 0.92,
 0.16666666666787933: 0.906,
 0.16666666666606034: 0.904

In [ ]:
visualization_set = torch.utils.data.dataset.Subset(valset, np.arange(500))
dataloader = torch.utils.data.DataLoader(visualization_set, batch_size=64, shuffle=False)

tensorboard_embeddings(model, ['max1'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_tripletreg')

tensorboard_embeddings(model, ['max2'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_tripletreg')

tensorboard_embeddings(model, ['max3'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_tripletreg')

tensorboard_embeddings(model, ['act1'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_tripletreg')

tensorboard_embeddings(model, ['act2'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_tripletreg')

tensorboard_embeddings(model, ['out'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_tripletreg')